In [ ]:
#Code for MJT nanowire and contact can have magantization in any direction

import numpy as np
import matplotlib.pyplot as plt
import time

'''Declearning Constand and Inputs =================='''
t1= time.time()
# Constants (all MKS, except energy which is in eV)
hbar = 1.06e-34
q = 1.6e-19

#Effective mass in lattice for insulator and ferromagnet
m = 0.18* 9.1e-31
mfm = 0.8*9.1e-31
IE = (q*q) / (2 * np.pi * hbar)
Ef = 2.25
kT = 0.025

#Band Energy of insulator with respect to ferromagnet
Ecc = 3.02

# Inpupt a =size of lattice
a = 0.25e-10

#t0, tm are values of t in channel and magnet repectivily (NOTE:for now we have asumme energy Ec is same in both)
t0 = (hbar**2) / (2 * m * (a**2) * q) 
tm = (hbar**2) / (2 * mfm * (a**2) * q)


#Lattic point in Source, channel and Drain
NS = 2
NC = 38
ND = 2
Np = NS + NC + ND


#Pauli's Matrix and rotating angle
theta = (0/180)*np.pi

ax = np.array([[0,1],[1,0]])
ay = np.array([[0,-1j],[1j,0]])
az = np.array([[1,0],[0,-1]])

n_hat = (np.sin(theta/2),0,np.cos(theta/2))

Rot = n_hat[0]*ax+n_hat[1]*ay+n_hat[2]*az

#Represent 'Extra energy or hamaltonian' due to ferromagnet
alpha = np.array([[2.15,0],[0,0]],dtype=complex)
beta = (np.conj(Rot.T))@alpha@Rot


'''Constructing Hamaltonian =========================-'''

diag0 = np.zeros(2*Np,dtype = complex)

for i in range((NS-1)):
    diag0[2*i] = 2*tm+alpha[0,0]
    diag0[2*i+1] = 2*tm + alpha[1,1]
    diag0[2*(Np-1-i)] = 2*tm+beta[0,0]
    diag0[2*(Np-i)-1] = 2*tm+beta[1,1]

for i in range((NC)):
    diag0[2*(NS+i)] = 2*t0 +Ecc
    diag0[2*(NS+i)+1] = 2*t0 +Ecc
    
    

#Terms to account for variable 
diag0[2*NS-2] = tm+t0+alpha[0,0]/2 +Ecc/2
diag0[2*NS-1] = tm+t0+ alpha[1,1]+Ecc/2
diag0[2*(Np-NS)] = tm+t0+beta[0,0]/2 +Ecc/2
diag0[2*(Np-NS)+1] = tm+t0+beta[1,1]/2 +Ecc/2

#Spin Hamaltonian off diagonal term
diag1= np.zeros(2*Np-1,dtype = complex)

for i in range(ND):
    diag1[2*Np-2-2*i]= beta[0,1]

diag_1 = np.conj(diag1)
    

#Off diagonal terms
diag2 = np.zeros(2*Np-2,dtype = complex)
diag2[:2*(NS-1)] = tm
diag2[2*(NS+NC):] = tm
diag2[2*(NS-1):2*(NS+NC)] = t0


T = np.diag(diag0) - np.diag(diag2, k=2) - np.diag(diag2, k=-2)+np.diag(diag1, k=1) + np.diag(diag_1, k=-1) 

#No barrier
UB = np.zeros(2*Np)

T = T + np.diag(UB)  

'''Applying Basis Votage    ================================='''    

# Bias
V_range = np.linspace(0, 10, 25) # Range of bias voltages to calculate
I_array = [] # Array to store the currents for each bias voltage

j =0;
for V in V_range:
    mu1 = Ef + (V/2)
    mu2 = Ef - (V/2)
    U1 = V * np.concatenate((0.5 * np.ones(2*NS), np.linspace(0.5, -0.5, 2*NC), -0.5 * np.ones(2*ND)))
    for i in range(NC):
        U1[2*(NS+i)+1]= U1[2*(NS+i)]

    '''Applying Green method at different Energy ============='''
    # Energy grid for Green's function method
    NE = 101
    E = np.linspace(mu2-15*kT, mu1+15*kT, NE)
    zplus = 1j * 1e-12
    dE = E[1] - E[0]
    f1 = 1 / (1 + np.exp((E - mu1) / kT))
    f2 = 1 / (1 + np.exp((E - mu2) / kT))

    # Transmission
    I = 0  # Current
    TM = np.zeros(NE)

    for k in range(NE):
        sig1 = np.zeros((2*Np, 2*Np), dtype=complex)
        sig2 = np.zeros((2*Np, 2*Np), dtype=complex)
        sig2_T = np.zeros((2*Np, 2*Np), dtype=complex)
        sig3 = np.zeros((2*Np, 2*Np), dtype=complex)
        #For contact 1
        ck_up = 1 - ((E[k] + zplus - U1[0] - UB[0]-alpha[0,0]) / (2 * tm))
        ka_up = np.arccos(ck_up)
        sig1[0, 0] = -tm * np.exp(1j*ka_up)
        ck_down = 1 - ((E[k] + zplus - U1[0] - UB[0]-alpha[1,1]) / (2 * tm))
        ka_down = np.arccos(ck_down)
        sig1[1, 1] = -tm * np.exp(1j*ka_down)
        gam1 = 1j * (sig1 - np.conj(sig1.T))
        
        #For contact 2
        ck_up = 1 - ((E[k] + zplus - U1[2*Np-2] - UB[2*Np-2]-alpha[0,0]) / (2 * tm))
        ka_up = np.arccos(ck_up)
        sig2_T[2*Np-2,2*Np-2] = -tm * np.exp(1j*ka_up)
        ck_down = 1 - ((E[k] + zplus - U1[2*Np-1] - UB[2*Np-1]-alpha[1,1]) / (2 * tm))
        ka_down = np.arccos(ck_down)
        
        #Rotating self energy matrix 
        sig2_T[2*Np-1,2*Np-1] = -tm * np.exp(1j*ka_down)
        sig2[2*Np-2:2*Np,2*Np-2:2*Np] = (np.conj(Rot.T))@sig2_T[2*Np-2:2*Np,2*Np-2:2*Np]@Rot
        gam2 = 1j * (sig2 - np.conj(sig2.T))
        
        G = ((E[k] + zplus) * np.eye(2*Np)) - T - np.diag(U1) - sig1 - sig2 - sig3
        TM[k] = np.real(np.trace(np.dot(np.dot(gam1, np.linalg.inv(G)), np.dot(gam2,np.conj((np.linalg.inv(G)).T)))))
        I = I + (dE * IE * TM[k] * (f1[k] - f2[k]))
    I_array.append(I)
    
        

t2 = time.time()


print(t2-t1);
plt.plot(V_range,I_array)
# Plot transmission vs energy

plt.show()

